In [2]:
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function

dataset = pd.read_csv("C:/Users/renan/Desktop/UFRGS/Machine Learning/Trabalho1/random-forests-1/datasets/benchmark.csv") 

dir(dataset)

print ('Banco com %d amostras e %d colunas\n' % dataset.shape)

print ('Tabela sumarizando as colunas do banco\n')
dataset

Banco com 14 amostras e 5 colunas

Tabela sumarizando as colunas do banco



,Tempo,Temperatura,Umidade,Ventoso,Joga
0,Ensolarado,Quente,Alta,Falso,Nao
1,Ensolarado,Quente,Alta,Verdadeiro,Nao
2,Nublado,Quente,Alta,Falso,Sim
3,Chuvoso,Amena,Alta,Falso,Sim
4,Chuvoso,Fria,Normal,Falso,Sim
5,Chuvoso,Fria,Normal,Verdadeiro,Nao
6,Nublado,Fria,Normal,Verdadeiro,Sim
7,Ensolarado,Amena,Alta,Falso,Nao
8,Ensolarado,Fria,Normal,Falso,Sim
9,Chuvoso,Amena,Normal,Falso,Sim


In [3]:
def bootstrap(dataset):

    """
    O comando sample da biblioteca pandas retorna uma amostra aleatória de itens.
    Recebe como parâmetro o dataset.
    'n' setado para manter o tamanho do dataset.
    'replace' habilitado para gerar uma amostra com reposição.
    """
    
    bootstrap = dataset.sample(n = len(dataset), replace = True)
    return bootstrap

teste1 = bootstrap(dataset)
teste1

,Tempo,Temperatura,Umidade,Ventoso,Joga
5,Chuvoso,Fria,Normal,Verdadeiro,Nao
1,Ensolarado,Quente,Alta,Verdadeiro,Nao
11,Nublado,Amena,Alta,Verdadeiro,Sim
12,Nublado,Quente,Normal,Falso,Sim
6,Nublado,Fria,Normal,Verdadeiro,Sim
12,Nublado,Quente,Normal,Falso,Sim
4,Chuvoso,Fria,Normal,Falso,Sim
11,Nublado,Amena,Alta,Verdadeiro,Sim
1,Ensolarado,Quente,Alta,Verdadeiro,Nao
2,Nublado,Quente,Alta,Falso,Sim


In [4]:
def bootstrap_resample(dataset):
    """ Bootstrap resample an array_like
    Parameters
    ----------
    X : array_like
      data to resample
    n : int, optional
      length of resampled array, equal to len(X) if n==None
    Results
    -------
    returns X_resamples
    """
    #if n == None:
    n = len(dataset)
        
    resample_i = np.floor(np.random.rand(n)*len(dataset)).astype(int)
    dataset_resample = dataset[resample_i]
    return dataset_resample

teste3 = bootstrap_resample(dataset)
teste3.describe()

KeyError: "None of [Int64Index([8, 4, 9, 3, 5, 1, 12, 13, 9, 9, 13, 6, 13, 4], dtype='int64')] are in the [columns]"

In [25]:
def k_folds(dataset, k=10):

    folds = np.array_split(dataset, k)          # Divide um array em múltiplos sub-arrays
    for i in range(k):
        train = folds.copy()                    # Cria uma cópia para cada fold
        train.pop(i)                            # Retira o item i do quadro.
        train = pd.concat(train, sort=False)    # Concatena os itens divididos em um conjunto de treinamento
        test = folds[i]                         # Os demais itens utilizados para treinamento
        yield train, test

# Teste da função 
for value in k_folds(dataset, k=10):  
    print(value) 

(         Tempo Temperatura Umidade     Ventoso Joga
2      Nublado      Quente    Alta       Falso  Sim
3      Chuvoso       Amena    Alta       Falso  Sim
4      Chuvoso        Fria  Normal       Falso  Sim
5      Chuvoso        Fria  Normal  Verdadeiro  Nao
6      Nublado        Fria  Normal  Verdadeiro  Sim
7   Ensolarado       Amena    Alta       Falso  Nao
8   Ensolarado        Fria  Normal       Falso  Sim
9      Chuvoso       Amena  Normal       Falso  Sim
10  Ensolarado       Amena  Normal  Verdadeiro  Sim
11     Nublado       Amena    Alta  Verdadeiro  Sim
12     Nublado      Quente  Normal       Falso  Sim
13     Chuvoso       Amena    Alta  Verdadeiro  Nao,         Tempo Temperatura Umidade     Ventoso Joga
0  Ensolarado      Quente    Alta       Falso  Nao
1  Ensolarado      Quente    Alta  Verdadeiro  Nao)
(         Tempo Temperatura Umidade     Ventoso Joga
0   Ensolarado      Quente    Alta       Falso  Nao
1   Ensolarado      Quente    Alta  Verdadeiro  Nao
4      Chuv

In [5]:
def stratified_k_fold(dataset, classe, k=10):
    data = dataset.sample(frac=1).reset_index(drop=True) # Shuffle DataFrame rows

    folds_per_class = []
    for _, g in data.groupby(class_column):
        folds_per_class.append(k_cross_fold(g, k=k))  # create fold for each class
    
    for _ in range(k):
        train = pd.DataFrame()
        test = pd.DataFrame()
        for c in range(len(folds_per_class)):         # append the next fold for each class
            train_c, test_c = next(folds_per_class[c])
            train = train.append(train_c)
            test = test.append(test_c)
        yield train, test

# Teste da função 
for value1 in stratified_k_fold(dataset, dataset[Joga], k=10):  
    print(value1) 

NameError: name 'Joga' is not defined

## Testes de avaliaçao para a arvore

In [9]:
from collections import Counter, defaultdict, OrderedDict
from functools import partial
import math, random
import pandas as pd
from csv import reader
import numpy as np
from random import seed
from random import randrange

- Gerar um banco de dados através do bootstrap
- Dividir o banco e K folds
- Separar os folds e dois bancos de dados, treino e teste
- aplicar os bancos de treinamento na arvore de decisão e fazer a classificação dos resultados com o banco de teste
- Avaliar os resultados em acuracia 

In [69]:
def bootstrap(dataset):
    bootstrap = dataset.sample(n = len(dataset), replace = True)
    return bootstrap

result_bootstrap = bootstrap(pd.read_csv('datasets/benchmark.csv'))

print(result_bootstrap)

         Tempo Temperatura Umidade     Ventoso Joga
1   Ensolarado      Quente    Alta  Verdadeiro  Nao
3      Chuvoso       Amena    Alta       Falso  Sim
1   Ensolarado      Quente    Alta  Verdadeiro  Nao
5      Chuvoso        Fria  Normal  Verdadeiro  Nao
6      Nublado        Fria  Normal  Verdadeiro  Sim
2      Nublado      Quente    Alta       Falso  Sim
9      Chuvoso       Amena  Normal       Falso  Sim
9      Chuvoso       Amena  Normal       Falso  Sim
8   Ensolarado        Fria  Normal       Falso  Sim
3      Chuvoso       Amena    Alta       Falso  Sim
9      Chuvoso       Amena  Normal       Falso  Sim
13     Chuvoso       Amena    Alta  Verdadeiro  Nao
3      Chuvoso       Amena    Alta       Falso  Sim
13     Chuvoso       Amena    Alta  Verdadeiro  Nao


In [75]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

cross_val = cross_validation_split(result_bootstrap.values.tolist(), 5)
cross_val

[[['Nublado', 'Fria', 'Normal', 'Verdadeiro', 'Sim'],
  ['Chuvoso', 'Amena', 'Normal', 'Falso', 'Sim']],
 [['Ensolarado', 'Fria', 'Normal', 'Falso', 'Sim'],
  ['Chuvoso', 'Amena', 'Alta', 'Verdadeiro', 'Nao']],
 [['Nublado', 'Quente', 'Alta', 'Falso', 'Sim'],
  ['Chuvoso', 'Amena', 'Alta', 'Falso', 'Sim']],
 [['Chuvoso', 'Amena', 'Alta', 'Verdadeiro', 'Nao'],
  ['Chuvoso', 'Amena', 'Alta', 'Falso', 'Sim']],
 [['Chuvoso', 'Amena', 'Normal', 'Falso', 'Sim'],
  ['Chuvoso', 'Amena', 'Normal', 'Falso', 'Sim']]]

In [92]:
def train_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        return train_set

def test_split(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    for fold in folds:
        test_set = list(fold)
        return test_set


# Não está retendo os dados do banco de bootstrap, é os mesmos dados, mas ainda tá flutuando e gerando listas diferentes, temos que discutir se isso tem algum problema 


test_set = test_split(cross_val, 5) 
print (test_set) #tem que remover a classe nesse banco

train_set = train_split(cross_val, 5) 
train_set

[[['Chuvoso', 'Amena', 'Alta', 'Verdadeiro', 'Nao'], ['Chuvoso', 'Amena', 'Alta', 'Falso', 'Sim']]]


[[['Nublado', 'Quente', 'Alta', 'Falso', 'Sim'],
  ['Chuvoso', 'Amena', 'Alta', 'Falso', 'Sim']],
 [['Chuvoso', 'Amena', 'Normal', 'Falso', 'Sim'],
  ['Chuvoso', 'Amena', 'Normal', 'Falso', 'Sim']],
 [['Ensolarado', 'Fria', 'Normal', 'Falso', 'Sim'],
  ['Chuvoso', 'Amena', 'Alta', 'Verdadeiro', 'Nao']],
 [['Chuvoso', 'Amena', 'Alta', 'Verdadeiro', 'Nao'],
  ['Chuvoso', 'Amena', 'Alta', 'Falso', 'Sim']]]

In [13]:
matrix = list()
accuracy = list()
recall = list()
precision = list()
score = list()

def classification_report(reais, preditos):
    # definindo as classes
    true_class = 1
    negative_class = 0
    # declarando a frêquencia
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    # criando a matriz de confusão
    for (indice, v_real) in enumerate(reais):
        v_predito = preditos[indice]
        # se trata de um valor real da classe positiva
        if v_real == true_class:
            tp += 1 if v_predito == v_real else 0
            fp += 1 if v_predito != v_real else 0
        else:
            tn += 1 if v_predito == v_real else 0
            fn += 1 if v_predito != v_real else 0
            
    # calculando os valores de frêquencia
    accuracy.append((tp+tn)/(tp+fp+tn+fn))
    recall.append(tp/(tp+fn))
    precision.append(tp/(tp+fp))
    score.append(2*tp/(2*tp+fp+fn))
    
    # desenhando a matriz de confusão
    print(np.array([
        [ tp, fp ],
        [ fn, tn ]
    ]))

In [14]:
# DIVISÃO TREINO E TESTE

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
